In [3]:
import kfp
from kfp import dsl

@dsl.pipeline(
    name='AttachStorage',
    description='Create a pvc, attach it to pipeline.'
)
def attatch_pvc_pipeline():
    
    vop = dsl.VolumeOp(
        name="volume_creation",
        resource_name="vol-a",
        storage_class="nfs-client",
        modes=dsl.VOLUME_MODE_RWM,
        size="1Gi"
    )
    
    cop1 = dsl.ContainerOp(
        name='HelloKubeflow',
        image='alpine',
        command=['sh', '-c'],
        arguments=['echo "hello Kubeflow" > /mnt/content.txt'],
        pvolumes={"/mnt": vop.volume}
    )
    ㅅ
    cop2 = dsl.ContainerOp(
        name='cat-content',
        image='alpine',
        command=['cat'],
        arguments=['/mnt/content.txt'],
        pvolumes={"/mnt": vop.volume}
    )

    cop1.after(vop)
    cop2.after(cop1)
        
     
if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(pipeline_func=attatch_pvc_pipeline, 
                                               arguments={})


In [8]:
!kubectl apply -f kubeflow-pvc2.yaml

persistentvolumeclaim/kubeflow-pvc-2 created
